### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
pd.reset_option("max_columns")
pd.set_option("max_colwidth",None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
#funcion para detectar columnas con valores NaN
def list_of_columns_function(df):
    x=[]
    for column in df.columns:
        if df[column].isna().any():
           x.append(column)
    return x
            

In [3]:
#funcion para visualizar la cantidad de nan por columnas
def visualization_of_nan(df):
    for column in df.columns:
        if df[column].isna().any():
            nan_count = df[column].isna().sum()
            print(f"Column '{column}' has {nan_count} NaN values.")
        else:
            print(f"Column `{column}´ has no NaN values.")  

In [4]:
df_actividad = pd.read_csv("actividadeconomica202309.csv",sep=";",low_memory=False,encoding="UTF-8")
#CSV de actividades comerciales en Madrid (extraido de una web publica-https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default-)

In [5]:
df_actividad= df_actividad.drop(columns=["fx_datos_ini","fx_datos_fin","id_local_agrupado","desc_tipo_agrup",\
                            "id_tipo_agrup","nombre_agrupacion","id_agrupacion","coordenada_y_agrupacion",\
                            "coordenada_x_agrupacion","cal_acceso","num_acceso","nom_acceso","id_clase_ndp_acceso",\
                            "id_ndp_acceso","id_vial_acceso","secuencial_local_PC","cal_edificio","num_edificio",\
                            "nom_edificio","id_ndp_edificio","desc_vial_edificio","clase_vial_edificio",\
                            "id_vial_edificio","desc_tipo_acceso_local","id_tipo_acceso_local","id_seccion_censal_local"])

"""
Eliminando features que no son necesarias para analisis
"""

df_licencias= pd.read_csv("licencias202309.csv",sep=";",encoding="UTF-8")#CSV de licencias en Madrid (extraido de una web publica-https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default-)
df_licencias.drop_duplicates(subset=['id_local'], keep='first', inplace=True)#Hay muchos locales que tienen mas de 1 licencia, un buen ejemplo son centros comerciales que tienen licencia de venta de alcohol y/o carniceria.
df_licencias= df_licencias[["id_local","ref_licencia","id_tipo_licencia","desc_tipo_licencia",\
                            "id_tipo_situacion_licencia","desc_tipo_situacion_licencia","Fecha_Dec_Lic"]]

"""
Extrayendo solo los features necesarios
"""

df_locales= pd.read_csv("locales202309.csv",sep=";",encoding="UTF-8")#CSV de locales comerciales en Madrid -https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
df_locales= df_locales[["id_local","desc_situacion_local","id_situacion_local","coordenada_x_local","coordenada_y_local","cod_barrio_local"]]

#seleccion de features para analisis y trabajo

df_terrazas= pd.read_csv("terrazas202309.csv",sep=";",encoding="UTF-8")#CSV de terrazas en Madrid -https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=cb784274d3c84410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
df_terrazas= df_terrazas[["id_terraza","id_local","id_periodo_terraza","desc_periodo_terraza",\
                        "id_situacion_terraza","desc_situacion_terraza","Superficie_ES",\
                         "desc_ubicacion_terraza","hora_ini_LJ_es","hora_fin_LJ_es","mesas_es","sillas_es"]]

In [6]:
merged_df= pd.merge(df_locales,df_licencias, how= "outer")
merged_df_2= pd.merge(merged_df,df_terrazas,how="outer",on="id_local")
merged_df_3= pd.merge(merged_df_2,df_actividad,how="outer",on="id_local")#merged final con todos los datasets y sus features seleccionados
df_cleaned= merged_df_3.loc[merged_df_3['id_division'] == "56"]#seleccion de locales por id_division=56 (restaurantes o establecimientos de alimentos y bebidas)

In [7]:
visualization_of_nan(df_cleaned)#uso de la funcion para contar NaN values

Column `id_local´ has no NaN values.
Column `desc_situacion_local_x´ has no NaN values.
Column `id_situacion_local_x´ has no NaN values.
Column `coordenada_x_local_x´ has no NaN values.
Column `coordenada_y_local_x´ has no NaN values.
Column `cod_barrio_local_x´ has no NaN values.
Column 'ref_licencia' has 5433 NaN values.
Column 'id_tipo_licencia' has 5433 NaN values.
Column 'desc_tipo_licencia' has 5433 NaN values.
Column 'id_tipo_situacion_licencia' has 5433 NaN values.
Column 'desc_tipo_situacion_licencia' has 5433 NaN values.
Column 'Fecha_Dec_Lic' has 5433 NaN values.
Column 'id_terraza' has 14869 NaN values.
Column 'id_periodo_terraza' has 14869 NaN values.
Column 'desc_periodo_terraza' has 14869 NaN values.
Column 'id_situacion_terraza' has 14869 NaN values.
Column 'desc_situacion_terraza' has 14869 NaN values.
Column 'Superficie_ES' has 14869 NaN values.
Column 'desc_ubicacion_terraza' has 14869 NaN values.
Column 'hora_ini_LJ_es' has 14869 NaN values.
Column 'hora_fin_LJ_es' 

In [8]:
print(len(df_cleaned))#total de observaciones en el dataset

21527


In [9]:
df_cleaned= df_cleaned.reset_index()

In [10]:
df_cleaned.head()

,index,id_local,desc_situacion_local_x,id_situacion_local_x,coordenada_x_local_x,coordenada_y_local_x,cod_barrio_local_x,ref_licencia,id_tipo_licencia,desc_tipo_licencia,id_tipo_situacion_licencia,desc_tipo_situacion_licencia,Fecha_Dec_Lic,id_terraza,id_periodo_terraza,desc_periodo_terraza,id_situacion_terraza,desc_situacion_terraza,Superficie_ES,desc_ubicacion_terraza,hora_ini_LJ_es,hora_fin_LJ_es,mesas_es,sillas_es,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local_y,desc_seccion_censal_local,coordenada_x_local_y,coordenada_y_local_y,id_situacion_local_y,desc_situacion_local_y,id_clase_ndp_edificio,clase_vial_acceso,desc_vial_acceso,id_planta_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,fx_carga
0,0,10001612,Abierto,1.0,441202.6,4473371.5,2.0,500/2015/06676,1.0,Declaración Responsable,3.0,Denegada,26/04/2017,1601.0,1.0,Anual,1.0,Abierta,11.7,Calle peatonal,09:00:00,23:30:00,6.0,16.0,1.0,CENTRO,102.0,EMBAJADORES,2.0,58.0,441202.6,4473371.5,1.0,Abierto,1.0,CALLE,DOCTOR DRUMEN,PB,SUKALDEA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,2023-09-08 07:00:48.237
1,7,10001663,Abierto,1.0,440387.6,4473231.5,2.0,350/2021/04565,4.0,Transmisión de licencia Urbanística,5.0,Transmisión de Licencia Concedida,01/01/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,102.0,EMBAJADORES,2.0,38.0,440387.6,4473231.5,1.0,Abierto,1.0,CALLE,MESON DE PAREDES,PB,AL YAOUHARA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237
2,12,10001692,Abierto,1.0,441034.6,4474069.5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2499.0,1.0,Anual,1.0,Abierta,36.3,Acera,10:00:00,01:00:00,17.0,34.0,1.0,CENTRO,103.0,CORTES,3.0,71.0,441034.6,4474069.5,1.0,Abierto,1.0,PLAZA,CANOVAS DEL CASTILLO,PB,STARBUCKS COFFEE,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,2023-09-08 07:00:48.237
3,13,10002228,Abierto,1.0,441032.6,4475090.5,4.0,350/2022/00813,4.0,Transmisión de licencia Urbanística,5.0,Transmisión de Licencia Concedida,01/01/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,104.0,JUSTICIA,4.0,84.0,441032.6,4475090.5,1.0,Abierto,1.0,CALLE,FERNANDO VI,PB,PIZZERIA-TRATTORIA TOTO E PEPPINO,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237
4,14,10002237,Abierto,1.0,439641.6,4473828.5,1.0,101/2003/15076,2.0,Licencia Urbanística,2.0,Concedida,22/12/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,101.0,PALACIO,1.0,11.0,439641.6,4473828.5,1.0,Abierto,1.0,PLAZA,PAJA,PB,NAIA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237


In [11]:
df_cleaned.coordenada_x_local_x.loc[df_cleaned["coordenada_x_local_x"]==0.00].count()#hay que crear coordenadas en funcion a la direccion de los locales, hay 2448 observaciones sin latitud y longitud


2448

In [12]:
df_cleaned.ref_licencia.isna().value_counts()#mas de 5k observaciones en nan, rellenar con none en vez de nan, ver la dispersion y localidades de los nan

ref_licencia
False    16094
True      5433
Name: count, dtype: int64

In [13]:
df_cleaned.id_tipo_licencia.isna().value_counts()#se repite el mismo patron que en referencia de licencia, colocar lo mismo que en referencia de licencia, se puede inferir?

id_tipo_licencia
False    16094
True      5433
Name: count, dtype: int64

In [14]:
len(df_cleaned)

21527

In [15]:
#creando una mascara para salir de los negocios que no son propiamente locales de hosteleria
mask =(df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS PARA MAYORES")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS EDUCATIVOS Y CENTROS DE CUIDADO INFANTIL")\
    | (df_cleaned["desc_epigrafe"] == "OTROS SERVICIOS DE COMIDAS EN INSTALACIONES DEPORTIVAS, OFICINAS, EMPRESAS O SIMILARES")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE COMEDOR EN CENTROS DE ACTIVIDADES DE SERVICIOS SOCIALES")\
    | (df_cleaned["desc_epigrafe"] == "SALONES DE BANQUETES Y PROVISION COMIDAS PARA EVENTOS")\
    | (df_cleaned["desc_epigrafe"] == "SERVICIOS DE PREPARACIÓN DE COMIDAS EN HOSPITALES")\
    | (df_cleaned["desc_epigrafe"] == "CIBER-CAFE")\
    | (df_cleaned["desc_epigrafe"] == "VENDEDOR AMBULANTE DE ALIMENTOS PREPARADOS PARA SU CONSUMO INMEDIATO")\


df_cleaned = df_cleaned[~mask]


In [16]:
len(df_cleaned)

19210

In [17]:
df_cleaned.head()

,index,id_local,desc_situacion_local_x,id_situacion_local_x,coordenada_x_local_x,coordenada_y_local_x,cod_barrio_local_x,ref_licencia,id_tipo_licencia,desc_tipo_licencia,id_tipo_situacion_licencia,desc_tipo_situacion_licencia,Fecha_Dec_Lic,id_terraza,id_periodo_terraza,desc_periodo_terraza,id_situacion_terraza,desc_situacion_terraza,Superficie_ES,desc_ubicacion_terraza,hora_ini_LJ_es,hora_fin_LJ_es,mesas_es,sillas_es,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local_y,desc_seccion_censal_local,coordenada_x_local_y,coordenada_y_local_y,id_situacion_local_y,desc_situacion_local_y,id_clase_ndp_edificio,clase_vial_acceso,desc_vial_acceso,id_planta_agrupado,rotulo,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe,fx_carga
0,0,10001612,Abierto,1.0,441202.6,4473371.5,2.0,500/2015/06676,1.0,Declaración Responsable,3.0,Denegada,26/04/2017,1601.0,1.0,Anual,1.0,Abierta,11.7,Calle peatonal,09:00:00,23:30:00,6.0,16.0,1.0,CENTRO,102.0,EMBAJADORES,2.0,58.0,441202.6,4473371.5,1.0,Abierto,1.0,CALLE,DOCTOR DRUMEN,PB,SUKALDEA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,2023-09-08 07:00:48.237
1,7,10001663,Abierto,1.0,440387.6,4473231.5,2.0,350/2021/04565,4.0,Transmisión de licencia Urbanística,5.0,Transmisión de Licencia Concedida,01/01/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,102.0,EMBAJADORES,2.0,38.0,440387.6,4473231.5,1.0,Abierto,1.0,CALLE,MESON DE PAREDES,PB,AL YAOUHARA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237
2,12,10001692,Abierto,1.0,441034.6,4474069.5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2499.0,1.0,Anual,1.0,Abierta,36.3,Acera,10:00:00,01:00:00,17.0,34.0,1.0,CENTRO,103.0,CORTES,3.0,71.0,441034.6,4474069.5,1.0,Abierto,1.0,PLAZA,CANOVAS DEL CASTILLO,PB,STARBUCKS COFFEE,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561006,CAFETERIA,2023-09-08 07:00:48.237
3,13,10002228,Abierto,1.0,441032.6,4475090.5,4.0,350/2022/00813,4.0,Transmisión de licencia Urbanística,5.0,Transmisión de Licencia Concedida,01/01/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,104.0,JUSTICIA,4.0,84.0,441032.6,4475090.5,1.0,Abierto,1.0,CALLE,FERNANDO VI,PB,PIZZERIA-TRATTORIA TOTO E PEPPINO,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237
4,14,10002237,Abierto,1.0,439641.6,4473828.5,1.0,101/2003/15076,2.0,Licencia Urbanística,2.0,Concedida,22/12/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CENTRO,101.0,PALACIO,1.0,11.0,439641.6,4473828.5,1.0,Abierto,1.0,PLAZA,PAJA,PB,NAIA,I,HOSTELERÍA,56,SERVICIOS DE COMIDAS Y BEBIDAS,561001,RESTAURANTE,2023-09-08 07:00:48.237


In [17]:
visualization_of_nan(df_cleaned)

Column `index´ has no NaN values.
Column `id_local´ has no NaN values.
Column `desc_situacion_local_x´ has no NaN values.
Column `id_situacion_local_x´ has no NaN values.
Column `coordenada_x_local_x´ has no NaN values.
Column `coordenada_y_local_x´ has no NaN values.
Column `cod_barrio_local_x´ has no NaN values.
Column 'ref_licencia' has 3905 NaN values.
Column 'id_tipo_licencia' has 3905 NaN values.
Column 'desc_tipo_licencia' has 3905 NaN values.
Column 'id_tipo_situacion_licencia' has 3905 NaN values.
Column 'desc_tipo_situacion_licencia' has 3905 NaN values.
Column 'Fecha_Dec_Lic' has 3905 NaN values.
Column 'id_terraza' has 12556 NaN values.
Column 'id_periodo_terraza' has 12556 NaN values.
Column 'desc_periodo_terraza' has 12556 NaN values.
Column 'id_situacion_terraza' has 12556 NaN values.
Column 'desc_situacion_terraza' has 12556 NaN values.
Column 'Superficie_ES' has 12556 NaN values.
Column 'desc_ubicacion_terraza' has 12556 NaN values.
Column 'hora_ini_LJ_es' has 12556 Na

In [18]:
df_cleaned= df_cleaned.drop(["fx_carga","id_epigrafe","id_clase_ndp_edificio",\
    "desc_seccion_censal_local","id_terraza","Fecha_Dec_Lic","cod_barrio_local_x",\
    "coordenada_x_local_y","coordenada_y_local_y","id_situacion_local_y",\
    "desc_situacion_local_y","hora_ini_LJ_es","hora_fin_LJ_es","desc_periodo_terraza",\
    "id_periodo_terraza","desc_tipo_licencia"], axis=1)

df_cleaned = df_cleaned.reindex(columns=["id_local", "rotulo", "desc_epigrafe","desc_division",\
    "desc_seccion","desc_situacion_local_x","id_situacion_local_x",\
    "id_tipo_licencia",\
    "id_tipo_situacion_licencia","desc_tipo_situacion_licencia",\
    "ref_licencia","id_situacion_terraza",\
    "desc_situacion_terraza","Superficie_ES","desc_ubicacion_terraza","mesas_es",\
    "sillas_es",'id_distrito_local', 'desc_distrito_local', "id_barrio_local",\
    "desc_barrio_local","cod_barrio_local_y",'clase_vial_acceso',\
    "desc_vial_acceso","id_planta_agrupado", "id_seccion","id_division",\
    "coordenada_y_local_x","coordenada_x_local_x"]).copy()



In [26]:
df_cleaned.ref_licencia.head()

0    500/2015/06676
1    350/2021/04565
2               NaN
3    350/2022/00813
4    101/2003/15076
Name: ref_licencia, dtype: object

In [34]:
for column in df_cleaned.columns:
    print(column)

id_local
rotulo
desc_epigrafe
desc_division
desc_seccion
desc_situacion_local_x
id_situacion_local_x
id_tipo_licencia
id_tipo_situacion_licencia
desc_tipo_situacion_licencia
ref_licencia
id_situacion_terraza
desc_situacion_terraza
Superficie_ES
desc_ubicacion_terraza
mesas_es
sillas_es
id_distrito_local
desc_distrito_local
id_barrio_local
desc_barrio_local
cod_barrio_local_y
clase_vial_acceso
desc_vial_acceso
id_planta_agrupado
id_seccion
id_division
coordenada_y_local_x
coordenada_x_local_x


In [35]:
len(df_cleaned.columns)

29

In [37]:
df_cleaned.head(2)

,id_local,rotulo,desc_epigrafe,desc_division,desc_seccion,desc_situacion_local_x,id_situacion_local_x,id_tipo_licencia,id_tipo_situacion_licencia,desc_tipo_situacion_licencia,ref_licencia,id_situacion_terraza,desc_situacion_terraza,Superficie_ES,desc_ubicacion_terraza,mesas_es,sillas_es,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local_y,clase_vial_acceso,desc_vial_acceso,id_planta_agrupado,id_seccion,id_division,coordenada_y_local_x,coordenada_x_local_x
0,10001612,SUKALDEA,CAFETERIA,SERVICIOS DE COMIDAS Y BEBIDAS,HOSTELERÍA,Abierto,1.0,1.0,3.0,Denegada,500/2015/06676,1.0,Abierta,11.7,Calle peatonal,6.0,16.0,1.0,CENTRO,102.0,EMBAJADORES,2.0,CALLE,DOCTOR DRUMEN,PB,I,56,4473371.5,441202.6
1,10001663,AL YAOUHARA,RESTAURANTE,SERVICIOS DE COMIDAS Y BEBIDAS,HOSTELERÍA,Abierto,1.0,4.0,5.0,Transmisión de Licencia Concedida,350/2021/04565,0.0,NaN,0.0,NaN,0.0,0.0,1.0,CENTRO,102.0,EMBAJADORES,2.0,CALLE,MESON DE PAREDES,PB,I,56,4473231.5,440387.6


In [48]:
df_cleaned.ref_licencia.isna().any()

False

In [40]:
visualization_of_nan(df_cleaned)

Column `id_local´ has no NaN values.
Column `rotulo´ has no NaN values.
Column `desc_epigrafe´ has no NaN values.
Column `desc_division´ has no NaN values.
Column `desc_seccion´ has no NaN values.
Column `desc_situacion_local_x´ has no NaN values.
Column `id_situacion_local_x´ has no NaN values.
Column `id_tipo_licencia´ has no NaN values.
Column `id_tipo_situacion_licencia´ has no NaN values.
Column `desc_tipo_situacion_licencia´ has no NaN values.
Column `ref_licencia´ has no NaN values.
Column `id_situacion_terraza´ has no NaN values.
Column 'desc_situacion_terraza' has 12556 NaN values.
Column `Superficie_ES´ has no NaN values.
Column 'desc_ubicacion_terraza' has 12556 NaN values.
Column `mesas_es´ has no NaN values.
Column `sillas_es´ has no NaN values.
Column `id_distrito_local´ has no NaN values.
Column `desc_distrito_local´ has no NaN values.
Column `id_barrio_local´ has no NaN values.
Column `desc_barrio_local´ has no NaN values.
Column `cod_barrio_local_y´ has no NaN values.


In [47]:
import utm

In [39]:
def hourglass(x, y):
    lat, lon = utm.to_latlon(x, y, 30, 'N')
    return pd.Series({'latitud': lat, 'longitud': lon})

df_defined[['latitud', 'longitud']] = df_defined.apply(lambda row: hourglass(row['longitud'], row['latitud']), axis=1)

df_defined

#df_defined.to_csv(f"censo_ac.csv", index=False)


OutOfRangeError: easting out of range (must be between 100,000 m and 999,999 m)

In [ ]:
map_madrid = folium.Map(location=[40.4168, -3.7038], zoom_start=14)

for (index, row) in plis.iterrows():
    folium.Marker(location = [row.loc["latitud"], row.loc["longitud"]],
    popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid
